In [ ]:
# arcgis_layer_to_geojson_pref_gdal.py
# 用法（示例）:
#   python arcgis_layer_to_geojson_pref_gdal.py \
#     --url "https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/Waymo_Service_Area_2025_03_03/FeatureServer/0" \
#     --out "waymo_atx_2025-03-03.geojson"
#
# 逻辑：
#  1) 优先尝试 GDAL：下载一次 EsriJSON 到临时文件 -> ogr2ogr 转 GeoJSON
#  2) 失败或无 GDAL -> 纯 Python：
#       2a) 分批（按 objectIds）直接请求 GeoJSON（f=geojson）
#       2b) 若不支持，改用 EsriJSON（f=json）并做最小几何转换（Polygon/Polyline/Point）

import argparse
import json
import shutil
import subprocess
import sys
import tempfile
from pathlib import Path
from typing import Dict, List

import requests

TIMEOUT = 60
CHUNK = 200      # 分批 objectIds 大小
OUT_SR = "4326"  # 输出坐标系 EPSG:4326
UA = "arcgis-to-geojson/1.0 (+https://example.org)"


def has_gdal() -> bool:
    return shutil.which("ogr2ogr") is not None


def fetch_object_ids(layer_url: str) -> List[int]:
    r = requests.get(
        f"{layer_url}/query",
        params={"where": "1=1", "returnIdsOnly": "true", "f": "json"},
        timeout=TIMEOUT,
        headers={"User-Agent": UA},
    )
    r.raise_for_status()
    ids = (r.json() or {}).get("objectIds") or []
    ids.sort()
    if not ids:
        raise RuntimeError("No objectIds returned. 请检查图层 URL 或访问权限。")
    return ids


def try_fetch_geojson_chunks(layer_url: str, ids: List[int], out_sr: str, chunk: int) -> List[Dict]:
    feats: List[Dict] = []
    for i in range(0, len(ids), chunk):
        part = ids[i:i + chunk]
        r = requests.get(
            f"{layer_url}/query",
            params={
                "objectIds": ",".join(map(str, part)),
                "outFields": "*",
                "outSR": out_sr,
                "f": "geojson",
            },
            timeout=TIMEOUT,
            headers={"User-Agent": UA},
        )
        r.raise_for_status()
        j = r.json()
        if not isinstance(j, dict) or j.get("type") != "FeatureCollection":
            raise RuntimeError("Server did not return GeoJSON.")
        feats.extend(j.get("features", []))
    if not feats:
        raise RuntimeError("Empty GeoJSON features from server.")
    return feats


def fetch_esri_chunks(layer_url: str, ids: List[int], out_sr: str, chunk: int) -> List[Dict]:
    feats: List[Dict] = []
    for i in range(0, len(ids), chunk):
        part = ids[i:i + chunk]
        r = requests.get(
            f"{layer_url}/query",
            params={
                "objectIds": ",".join(map(str, part)),
                "outFields": "*",
                "outSR": out_sr,
                "f": "json",
            },
            timeout=TIMEOUT,
            headers={"User-Agent": UA},
        )
        r.raise_for_status()
        j = r.json()
        feats.extend(j.get("features", []))
    if not feats:
        raise RuntimeError("Empty EsriJSON features from server.")
    return feats


def esri_geom_to_geojson(esri_geom: Dict) -> Dict | None:
    if not esri_geom:
        return None
    if "rings" in esri_geom:  # Polygon
        return {"type": "Polygon", "coordinates": esri_geom["rings"]}
    if "paths" in esri_geom:  # Polyline
        paths = esri_geom["paths"]
        if len(paths) == 1:
            return {"type": "LineString", "coordinates": paths[0]}
        return {"type": "MultiLineString", "coordinates": paths}
    if "x" in esri_geom and "y" in esri_geom:  # Point
        return {"type": "Point", "coordinates": [esri_geom["x"], esri_geom["y"]]}
    return None  # 复杂几何可按需扩展


def esri_features_to_fc(esri_feats: List[Dict]) -> Dict:
    fc = {"type": "FeatureCollection", "features": []}
    for f in esri_feats:
        gj_geom = esri_geom_to_geojson(f.get("geometry", {}))
        if gj_geom is None:
            continue
        props = f.get("attributes", {}) or {}
        fc["features"].append({"type": "Feature", "geometry": gj_geom, "properties": props})
    return fc


def save_fc(features: List[Dict], out_path: Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as f:
        json.dump({"type": "FeatureCollection", "features": features}, f, ensure_ascii=False)
    print(f"[OK] Saved GeoJSON -> {out_path} | features={len(features)}")


def gdal_convert_from_esrijson_bytes(esri_bytes: bytes, out_path: Path) -> bool:
    """
    把 EsriJSON 字节写入临时文件，调用 ogr2ogr 转为 GeoJSON。
    返回 True/False 表示是否成功。
    """
    if not has_gdal():
        return False

    with tempfile.TemporaryDirectory() as td:
        tmp_esri = Path(td) / "layer.esrijson"
        tmp_esri.write_bytes(esri_bytes)

        out_path.parent.mkdir(parents=True, exist_ok=True)
        cmd = ["ogr2ogr", "-f", "GeoJSON", str(out_path), str(tmp_esri), "-overwrite"]
        try:
            print("[GDAL] Running:", " ".join(cmd))
            cp = subprocess.run(cmd, check=True, capture_output=True, text=True)
            print(cp.stdout.strip() or "[GDAL] done.")
        except subprocess.CalledProcessError as e:
            print("[GDAL] ogr2ogr failed:", e.stderr.strip() if e.stderr else e)
            return False

    # 简单校验
    try:
        j = json.loads(out_path.read_text(encoding="utf-8"))
        if j.get("type") == "FeatureCollection" and len(j.get("features", [])) > 0:
            return True
    except Exception:
        pass
    return False


def main():
    ap = argparse.ArgumentParser(description="Download ArcGIS FeatureServer layer as GeoJSON (prefer GDAL, fallback to pure Python).")
    ap.add_argument("--url", required=True, help="ArcGIS FeatureServer layer URL, e.g. .../FeatureServer/0")
    ap.add_argument("--out", required=True, help="Output GeoJSON file path")
    ap.add_argument("--chunk", type=int, default=CHUNK, help="Chunk size for objectIds")
    ap.add_argument("--out-sr", default=OUT_SR, help="Output spatial reference (e.g., 4326)")
    args = ap.parse_args()

    layer_url = args.url.rstrip("/")
    out_path = Path(args.out)

    # ---------- 方案 A：GDAL ----------
    if has_gdal():
        try:
            # 用一次性 EsriJSON（整层），多数服务区图层规模很小，直接拉取即可
            print("[GDAL] ogr2ogr detected. Trying GDAL path...")
            r = requests.get(
                f"{layer_url}/query",
                params={"where": "1=1", "outFields": "*", "outSR": args.out_sr, "f": "json"},
                timeout=TIMEOUT,
                headers={"User-Agent": UA},
            )
            r.raise_for_status()
            ok = gdal_convert_from_esrijson_bytes(r.content, out_path)
            if ok:
                print("[GDAL] Conversion succeeded.")
                return
            else:
                print("[GDAL] Conversion failed, falling back to pure Python...")
        except Exception as e:
            print(f"[GDAL] Path failed: {e}. Falling back to pure Python...")

    # ---------- 方案 B：纯 Python ----------
    try:
        print("[PY] Fetching objectIds...")
        ids = fetch_object_ids(layer_url)
        print(f"[PY] objectIds count = {len(ids)}")

        # 先尝试 GeoJSON
        try:
            print("[PY] Trying server-side GeoJSON...")
            feats_gj = try_fetch_geojson_chunks(layer_url, ids, args.out_sr, args.chunk)
            save_fc(feats_gj, out_path)
            return
        except Exception as e:
            print(f"[PY] GeoJSON not available, reason: {e}")

        # 回退 EsriJSON -> GeoJSON
        print("[PY] Falling back to EsriJSON -> GeoJSON conversion...")
        esri_feats = fetch_esri_chunks(layer_url, ids, args.out_sr, args.chunk)
        fc = esri_features_to_fc(esri_feats)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        with out_path.open("w", encoding="utf-8") as f:
            json.dump(fc, f, ensure_ascii=False)
        print(f"[OK] Saved GeoJSON (converted) -> {out_path} | features={len(fc['features'])}")

    except Exception as e:
        print("[ERR] Failed to export layer:", e)
        sys.exit(1)


if __name__ == "__main__":
    main()
